In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import tensorflow_datasets as tfds
from collections import Counter
import scipy.ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import gc
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tarfile
import urllib.request
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F
from scipy.stats import gaussian_kde
from numba import jit
from torchvision.datasets import Places365
from tqdm import tqdm  # 導入進度條庫
import json


os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
torch.backends.cudnn.benchmark = True
limit = 1000

# bzq modifying
len_x_target = 20
len_y_target = 20
stride_x_target = 10
stride_y_target = 10

# mean, std proportion
alpha = 1

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 0
pixels = 0

In [ ]:
# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to tensor (values in range 0–1)
    transforms.Lambda(lambda x: x * 255.0),  # Scale to range 0–255
    transforms.Lambda(lambda x: x[[2, 1, 0], ...]),  # Convert RGB to BGR
])

# 建立隨機影像數據集（不依賴 Places365）
class RandomImageDataset(torch.utils.data.Dataset):
    def __init__(self, num_images=1000, image_size=(224, 224)):
        self.num_images = num_images
        self.image_size = image_size

    def __len__(self):
        return self.num_images

    def __getitem__(self, idx):
        random_image = np.random.randint(0, 256, (self.image_size[1], self.image_size[0], 3), dtype=np.uint8)
        image = Image.fromarray(random_image)
        return transform(image)

# 使用自定義隨機影像數據集
dataset = RandomImageDataset(num_images=limit)

In [ ]:

def adjust_learning_rate(optimizer, epoch, initial_lr):
    if epoch <= 5:
        lr = initial_lr * epoch / 5
    elif epoch > 160:
        lr = initial_lr * 0.01
    elif epoch > 75:
        lr = initial_lr * 0.1
    else:
        lr = initial_lr

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Define the VGG16 model
model = models.vgg16(weights=False)  # Initialize without pre-trained weights
model = model.to(device)
num_features = model.classifier[6].in_features  # Get the input features of the last FC layer
model.classifier[6] = torch.nn.Linear(num_features, 365)  # Modify the last layer for 365 classes

model.features = model.features.to(device)
model.classifier = model.classifier.to(device)


# 定義損失函數和優化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
num_epochs = 200

if os.path.exists(f"vgg16_places365.pt"):
    # 加載保存的權重
    checkpoint = torch.load("vgg16_places365.pt")

    # Mapping Caffe keys to PyTorch keys
    mapped_state_dict = {
        # Conv1 Layers
        "features.0.weight": checkpoint["conv1_1.weight"],
        "features.0.bias": checkpoint["conv1_1.bias"],
        "features.2.weight": checkpoint["conv1_2.weight"],
        "features.2.bias": checkpoint["conv1_2.bias"],
        
        # Conv2 Layers
        "features.5.weight": checkpoint["conv2_1.weight"],
        "features.5.bias": checkpoint["conv2_1.bias"],
        "features.7.weight": checkpoint["conv2_2.weight"],
        "features.7.bias": checkpoint["conv2_2.bias"],
        
        # Conv3 Layers
        "features.10.weight": checkpoint["conv3_1.weight"],
        "features.10.bias": checkpoint["conv3_1.bias"],
        "features.12.weight": checkpoint["conv3_2.weight"],
        "features.12.bias": checkpoint["conv3_2.bias"],
        "features.14.weight": checkpoint["conv3_3.weight"],
        "features.14.bias": checkpoint["conv3_3.bias"],
        
        # Conv4 Layers
        "features.17.weight": checkpoint["conv4_1.weight"],
        "features.17.bias": checkpoint["conv4_1.bias"],
        "features.19.weight": checkpoint["conv4_2.weight"],
        "features.19.bias": checkpoint["conv4_2.bias"],
        "features.21.weight": checkpoint["conv4_3.weight"],
        "features.21.bias": checkpoint["conv4_3.bias"],
        
        # Conv5 Layers
        "features.24.weight": checkpoint["conv5_1.weight"],
        "features.24.bias": checkpoint["conv5_1.bias"],
        "features.26.weight": checkpoint["conv5_2.weight"],
        "features.26.bias": checkpoint["conv5_2.bias"],
        "features.28.weight": checkpoint["conv5_3.weight"],
        "features.28.bias": checkpoint["conv5_3.bias"],
        
        # Fully Connected Layers
        "classifier.0.weight": checkpoint["fc6.weight"],
        "classifier.0.bias": checkpoint["fc6.bias"],
        "classifier.3.weight": checkpoint["fc7.weight"],
        "classifier.3.bias": checkpoint["fc7.bias"],
        "classifier.6.weight": checkpoint["fc8a.weight"],
        "classifier.6.bias": checkpoint["fc8a.bias"],
    }

    # Load the updated state dict into the model
    model.load_state_dict(mapped_state_dict)

    # 切換模型到評估模式
    model.eval()
else:
    # 獲取數據
    image, target = dataset[0]
    print(f"Image size: {image.size()}, Target: {target}")
    
    # 計算每個類別的樣本數
    targets = [sample[1] for sample in dataset]  # 假設 dataset 返回 (data, label)
    class_counts = torch.bincount(torch.tensor(targets))
    class_weights = 1.0 / class_counts.float()
    weights = torch.tensor([class_weights[label] for label in targets])

    # 創建 WeightedRandomSampler
    sampler = torch.utils.data.WeightedRandomSampler(
        weights=weights, 
        num_samples=len(weights), 
        replacement=True
    )

    train_loader = DataLoader(dataset, batch_size=80, sampler=sampler, shuffle=False, num_workers=6, pin_memory=True)
    val_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)
    print("ready")
    
    scaler = torch.amp.GradScaler()

    for epoch in range(num_epochs):
        adjust_learning_rate(optimizer, epoch, 0.1)

        model.train()  # 設定模型為訓練模式
        running_loss = 0.0
        batch_count = 0

        # 使用 tqdm 包裹 DataLoader，顯示進度條
        with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as progress_bar:
            for images, labels in progress_bar:
                images, labels = images.to(device), labels.to(device)

                # 混合精度前向傳播
                with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                # 縮放損失並進行反向傳播
                scaler.scale(loss).backward()

                # 更新參數
                scaler.step(optimizer)
                scaler.update()

                # 清零梯度
                optimizer.zero_grad()

                running_loss += loss.item()
                batch_count += 1
                # 在進度條中顯示當前損失
                progress_bar.set_postfix(loss=f"{running_loss / batch_count:.4f}")

        # 評估準確度
        model.eval()  # 設定模型為評估模式
        correct = 0
        total = 0
        
        with torch.no_grad():  # 停用梯度計算
            for images, labels in val_loader:  
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)  # 獲取預測標籤
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch + 1}, Accuracy: {accuracy:.2f}%")

        # 保存模型權重
        torch.save(model.state_dict(), f"Places365_vgg16_epoch_{epoch + 1}.pth")
        print(f"Model saved for epoch {epoch + 1}")

    print("Training complete!")


In [ ]:
# 得到了bzq 正確的函數，拿來做imagenet 正確的預測

def single_data_bzq_mask_preprocessing_imagenet(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x, :] *= magnification
    return new_data


#print(random_num_for_bzq_mask_imagenet)

def single_data_bzq_mask_preprocessing_imagenet_random_global(original_data, start_x, start_y, len_x, len_y, random_num_for_bzq_mask_imagenet):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    random_num_for_bzq_mask_imagenet = random_num_for_bzq_mask_imagenet[:, :len_y, :len_x] 
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] = random_num_for_bzq_mask_imagenet
    return new_data


bzq = []
correct_predictions_imagenet = []
incorrect_predictions_imagenet = []
bzq_imagenet = []


#bzq = 0的時候，提取mask之下的softmax
correct_predictions_bzq_zero_softmax_mean = []
correct_predictions_bzq_zero_softmax_std = []


In [ ]:
##################################################################################################################################
def reverse_normalize(image, mean, std):
    # 確保均值和標準差是 numpy array，以支持逐像素計算
    mean = np.array(mean).reshape(1, 1, -1)  # [1, 1, C]
    std = np.array(std).reshape(1, 1, -1)    # [1, 1, C]

    # 反標準化公式
    image = (image * std) + mean
    return np.clip(image, 0, 255)  # 確保範圍在 [0, 255]

def normalize(image, mean, std):
    # 將影像標準化
    mean = np.array(mean).reshape(1, 1, -1)  # [1, 1, C]
    std = np.array(std).reshape(1, 1, -1)    # [1, 1, C]
    image = (image - mean) / std
    return image

    
test_dataset = dataset
image = test_dataset[0]
print(image.shape)


# 創建 DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)

In [ ]:
conf_vanilla = []
# 預測並顯示分數

correct = 0
total = 0
img = test_dataset[0]

print(len(test_dataset))

# 顯示影像
plt.imshow(np.array(img).transpose((1, 2, 0)) / 255.0)
plt.axis("off")  # 隱藏座標軸
plt.title("Sample Image")  # 標題，可選
plt.show()

In [ ]:
# bzq modifying
len_x = len_x_target
len_y = len_y_target
stride_x = stride_x_target
stride_y = stride_y_target
batch_size = 1  # 設定批次大小

original_predictions_imagenet = []

with torch.no_grad():
    for batch_data in test_loader:
        batch_data = batch_data.to(device).float()
        batch_predictions_imagenet = model(batch_data).cpu().numpy()
        original_predictions_imagenet.append(batch_predictions_imagenet)

original_predictions_imagenet = np.vstack(original_predictions_imagenet)

# 使用正確的方式來訪問 test_dataset 中的標籤
test_labels = [test_dataset[i][1] for i in range(len(test_dataset))]

predicted_labels = np.argmax(original_predictions_imagenet, axis=1)

In [ ]:
import torch
import numpy as np
from collections import Counter
import torch.nn.functional as F

random_num_for_bzq_mask_imagenet = np.random.randint(0, 256, (3, len_y_target, len_x_target)).astype(np.float32)

def process_batch(batch_data, len_y, stride_y, len_x, stride_x, device, model, alpha, bzq, entropy_values):
    if len(batch_data) == 0:
        return bzq, np.array([])

    single_data_bzq_classification_record = []
    targets = []

    for i in range(0, 224 - len_y, stride_y):
        for j in range(0, 224 - len_x, stride_x):
            target = single_data_bzq_mask_preprocessing_imagenet(batch_data, i, j, len_x, len_y, 0)
            # target = single_data_bzq_mask_preprocessing_imagenet_random_global(batch_data, i, j, len_x, len_y, random_num_for_bzq_mask_imagenet)
            targets.append(target)
    
    targets_tensor = torch.from_numpy(np.vstack(targets).reshape(-1, 3, 224, 224)).float().to(device)
    # 執行模型推論
    with torch.no_grad():
        predictions = model(targets_tensor)
        softmax_predictions = F.softmax(predictions, dim=1)
    
    #temp, _ = torch.max(F.softmax(predictions), dim=1)
    #temp = temp.cpu().numpy()

    max_bzq_indices = torch.argmax(predictions, dim=1).cpu().numpy()

    counter = Counter(max_bzq_indices)
    most_common_num, most_common_count = counter.most_common(1)[0]
    
    temp = softmax_predictions[:, most_common_num].cpu().numpy()

    bzq.append(alpha * np.mean(temp) + (1 - alpha) * (2.0 / np.pi * np.arctan(1.0 / np.std(temp))))

    # **計算熵**
    entropy = (-softmax_predictions * torch.log(softmax_predictions + 1e-8)).sum(dim=1).cpu().numpy()
    entropy_values.append(np.mean(entropy))

    return bzq, temp, entropy_values

entropy = []

with torch.no_grad():
    for start in range(0, len(test_dataset), batch_size):
        end = min(start + batch_size, len(test_dataset))
        batch_data = [test_dataset[i] for i in range(start, end)]
        
        for k in range(len(batch_data)):
            if len(batch_data[k]) > 0:
                entropy_values = []
                bzq, temp, entropy_values = process_batch(batch_data[k], len_y, stride_y, len_x, stride_x, device, model, alpha, bzq, entropy_values)
                original_data = single_data_bzq_mask_preprocessing_imagenet(batch_data[k], 0, 0, 0, 0, 0)
                original_prediction = model(torch.tensor(original_data.reshape(-1, 3, 224, 224)).float().to(device))
                max_original_index = torch.argmax(original_prediction).item()
                
                entropy.append(entropy_values)
                
                correct_predictions_bzq_zero_softmax_mean.append(np.mean(temp))
                correct_predictions_bzq_zero_softmax_std.append(np.std(temp))

                

bzq_imagenet = np.array(bzq)


In [ ]:
entropy = np.array(entropy).flatten()

# 計算熵的分佈（分桶）
bins = np.linspace(min(entropy), max(entropy), 15) 

hist, bin_edges = np.histogram(entropy, bins=bins)  # 計算直方圖數據
print(bin_edges[:-1], hist)
# 轉換為折線圖
plt.figure(figsize=(10, 5))
plt.plot(bin_edges[:-1], hist, linestyle='-', marker='o', color='red', label="BZQ")

# 設定標題與軸標籤
plt.title("Entropy")
plt.xlabel("Entropy")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.show()